In [ ]:
import pywatershed
import pandas as pd
from pathlib import Path as pl
import json
import numpy as np
from pywatershed.parameters.prms_parameters import JSONParameterEncoder

import sys
sys.path.append('../scripts/')
from pest_utils import pars_to_tpl_entries
sys.path.append('../dependencies/')
import pyemu

## Functions
### Creates a function that writes the parameter file as a json file

In [ ]:
def write_to_json_tpl(dims, pars, json_filename):
    with open(json_filename, "w") as ofp:
        ofp.write('ptf ~\n')
        json.dump(
            {**dims,
            **pars},
            ofp,
            indent=4,
            cls=JSONParameterEncoder,
        )
    # this sucks - should be a more direct way but whatevs. it verks
    inlines = open(json_filename, 'r').readlines()
    with open(json_filename, 'w') as ofp:
        [ofp.write(i.replace('"~','~').replace('~"','~')) for i in inlines]

## We will be running on each cutout eventually, but start with one

In [ ]:
wkdir = pl('../NHM_extractions/20230110_pois_haj/05431486/')

## Read in the oldskool parameter file as nested dictionaries, a.k.a. a Json-style file. This file would need to be written as a .txt file like the original myparam.param, unless Joe changes the pyWatershed code to read in the json-style file instead. Check with Mike/Joe.
## Eddie and Andy changed the path for this read to the "starting values" that Parker gave us for each extraction.

In [ ]:
wkdir

In [ ]:
####This was made in a previous notebook now
#pardat = pywatershed.parameters.PrmsParameters.load(wkdir / "myparam.param")#load parameter file from extraction
#pardat.parameters_to_json(wkdir /"parameters.json")


## Now the upper portion making the jason and the lower blocks running the model should be completed prior to the output notebook..or in it.

In [ ]:
pardat = pywatershed.parameters.PrmsParameters.load(wkdir / "parameters.json")#load parameter file from extraction
pars = pardat.parameters
#pars

In [ ]:
dims = pardat.dimensions
dims

In [ ]:
#Lets check for the pars needed to run NHM model using pyWatershed

In [ ]:
nhm_processes = [
    pywatershed.PRMSSolarGeometry,
    pywatershed.PRMSAtmosphere,
    pywatershed.PRMSCanopy,
    pywatershed.PRMSSnow,
    pywatershed.PRMSRunoff,
    pywatershed.PRMSSoilzone,
    pywatershed.PRMSGroundwater,
    pywatershed.PRMSChannel,
]

nhm_params = []
for proc in nhm_processes:
    nhm_params += proc.get_parameters()

In [ ]:
nhm_params

#### View the keys for "pars" --These will be the parameters with values listed in the "myparam_starting_vals.param" file of the extraction.

In [ ]:
pars.keys()

#### View values of one paramenter

In [ ]:
pars['nhm_id']

## Create a PEST template file version of json-style of myparam_starting_vals.param, "pars"

In [ ]:
# Make a list of hrus from "pars"
hrus = pars['nhm_id']
hrus

In [ ]:
segs = pars['nhm_seg']
segs

### Run through all the currently-defined parameters, and using Mike's function "pars_to_tpl_entries()", write param stating values to a new dataframe "par_starting_vals"

In [ ]:
# Create empty dataframe with columns parname (pestpp param name) and parval1 (pestpp starting value)
par_starting_vals = pd.DataFrame(columns=['parname','parval1', 'parubnd','parlbnd'])

In [ ]:
par_starting_vals

In [ ]:
par_starting_vals = pars_to_tpl_entries(pars, 'adjmix_rain', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'carea_max', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'cecn_coef', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'emis_noppt', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'fastcoef_lin', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'freeh2o_cap', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'gwflow_coef', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'jh_coef', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'mann_n', hrus, segs, par_starting_vals, hru_based=False, 
                    seg_based=True, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'potet_sublim', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'rad_trncf', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'radmax', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'rain_cbh_adj', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'slowcoef_sq', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'smidx_coef', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'snarea_thresh', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'snowinfil_max', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'snow_cbh_adj', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'soil2gw_max', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'soil_moist_max', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'soil_rechr_max_frac', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'ssr2gw_exp', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'ssr2gw_rate', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=False)
par_starting_vals = pars_to_tpl_entries(pars, 'tmax_allrain_offset', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'tmax_allsnow', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'tmax_cbh_adj', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)
par_starting_vals = pars_to_tpl_entries(pars, 'tmin_cbh_adj', hrus, segs, par_starting_vals, hru_based=True, 
                    seg_based=False, month_based=True)


In [ ]:
par_starting_vals

In [ ]:
par_starting_vals.set_index('parname', inplace =True, drop = False)
par_starting_vals

In [ ]:
#Set values for upper and lower bounds


### once we have all the parameter arrays replaced by names, we can write out the template file

In [ ]:
write_to_json_tpl(dims, pars, wkdir / 'parameters.json.tpl')
par_starting_vals.to_csv(wkdir / 'starting_par_vals.dat', index=None, sep=' ')

### Map observation name to the Instruction File (.ins)

In [ ]:
obsvals = pd.read_csv(wkdir / 'allobs.dat', delim_whitespace= True)
obsvals.set_index('obsname', inplace =True, drop = False)
#obsvals.sample(5)
print(obsvals)
print('The values for "obsval" are the true observation values.')

In [ ]:
with open(wkdir /'modelobs.dat.ins', 'w') as ofp:
    ofp.write('pif ~\n')
    ofp.write('~obsval~\n')
    [ofp.write(f'l1 w !{i}!\n') for i in obsvals.obsname]

### create PST control file object with `pyemu`

In [ ]:
pst = pyemu.Pst.from_io_files(tpl_files = [str(wkdir / 'parameters.json.tpl')],
                              in_files=[str(wkdir / 'parameters.json')],# Values for parval1 and bnds will be populated with default values
                              ins_files = [str(wkdir / 'modelobs.dat.ins')],
                              out_files = [str(wkdir / 'modelobs.dat')], #names the model output file in the control file (prior_mc.pst)--Chk with Mike
                              pst_path = '.')
#Ask Mike if pyemu reads in the values for the obs from the modelobs.dat file

## Direct editing of the PEST parameter file

In [ ]:
pars = pst.parameter_data
#pars
#Note the param.param file contained values calibrated already, so we will replace parval1 with precal starting values

In [ ]:
pars

In [ ]:
#pars.loc['adjmix_rain:hru_5621:mon_1','parval1'] = 987236
#pst.parameter_data


### Copy parval1 from "par_starting_vals" to pars.parval1 

In [ ]:
for idx, row in pars.iterrows():
    pars.loc[pars.parnme,'parval1'] = par_starting_vals.loc[pars.parnme,'parval1']


In [ ]:
pars

### Copy upper and lower bounds from par_cal_bounds_use.csv to par.parubnd and par.parlbnd

In [ ]:
bnds_path = '../Supporting_information/par_cal_bounds_use.csv'
bnds = pd.read_csv(bnds_path) # Creates a data frame of the bounds for par catagories

In [ ]:
bnds.set_index('parameter_name', inplace =True, drop = False)
#bnds

In [ ]:
prms_parnme_list =bnds['parameter_name']# Make a list of the nhm par names for loops below

In [ ]:
for idx, row in pars.iterrows():
    for i in prms_parnme_list:
        pst_parnme = str(row.parnme)
        prms_parnme = prms_parnme_list[i]
        x = pst_parnme.startswith(prms_parnme)# Just a yes not response to if the pst parname starts with the root in "".
        if x :
            pars.loc[pst_parnme,'parubnd'] = bnds.loc[prms_parnme,'par_upper_bound']
            pars.loc[pst_parnme,'parlbnd'] = bnds.loc[prms_parnme,'par_lower_bound']

### we can't log transform negative parameter values

In [ ]:
pars.loc[pars.parlbnd<=0, 'partrans'] = 'none'

In [ ]:
### obs.loc[obsvals.obsname, 'obsval'] = obsvals.obsval.values

#### Set obsval in the "pst.observation_data" frame back to the true observation value

In [ ]:
obs = pst.observation_data #This pulls the "observation data" from the pst dataframe and sets it to the "obs" object (dataframe)

In [ ]:
#obs.loc[obs.obsnme, :]

In [ ]:
obs.loc[obs.obsnme =='actet_mon:2000_1:5621',:] # This is the value in the modelobs.dat file?

In [ ]:
obsvals.loc[obsvals.obsname =='actet_mon:2000_1:5621',:]

In [ ]:
#obs = obs.loc[obsvals.obsname,:] #resorts datframe for easy in reading

In [ ]:
obs.loc[obsvals.obsname, 'obsval'] = obsvals.obsval.values #True observation value is copied over to obs
obs

In [ ]:
obs.loc[obs.obsnme =='actet_mon:2000_1:5621',:] # Check for change

In [ ]:
#Set all weights for no data days in obs to "0."
obs.loc[obs.obsval==-9999,'weight'] = 0
obs

#### Creating Groups observations

In [ ]:
obs.loc[obs.obsnme.str.startswith('actet_mon'),'obgnme'] = 'actet_mon'
obs.loc[obs.obsnme.str.startswith('actet_mean_mon'),'obgnme'] = 'actet_mean_mon'
obs.loc[obs.obsnme.str.startswith('recharge_ann'),'obgnme'] = 'recharge_ann'
obs.loc[obs.obsnme.str.startswith('soil_moist_mon'),'obgnme'] = 'soil_moist_mon'
obs.loc[obs.obsnme.str.startswith('soil_moist_ann'),'obgnme'] = 'soil_moist_ann'
obs.loc[obs.obsnme.str.startswith('runoff_mon'),'obgnme'] = 'runoff_mon'
obs.loc[obs.obsnme.str.startswith('sca_daily'),'obgnme'] = 'sca_daily'
#obs.loc[obs.obsnme.str.startswith('streamflow_daily'),'obgnme'] = 'streamflow_daily'

# Create EFC Groups for daily streamflows
# streamflow_daily is followed by a suffix: "efc"_"high_low" integers
# efc [1, 2, 3, 4, 5] are ['Large flood', 'Small flood', 'High flow pulse', 'Low flow', 'Extreme low flow']
# high_low [1, 2, 3] are ['Low flow', 'Ascending limb', 'Descending limb']
# Pest++ group names were written with flows in mind.

obs.loc[obs.obsnme.str.startswith('streamflow_daily_1_2'),'obgnme'] = 'streamflow_daily_large_asc'
obs.loc[obs.obsnme.str.startswith('streamflow_daily_1_3'),'obgnme'] = 'streamflow_daily_large_dsc'
obs.loc[obs.obsnme.str.startswith('streamflow_daily_2_2'),'obgnme'] = 'streamflow_daily_small_asc'
obs.loc[obs.obsnme.str.startswith('streamflow_daily_2_3'),'obgnme'] = 'streamflow_daily_small_dsc'
obs.loc[obs.obsnme.str.startswith('streamflow_daily_3_2'),'obgnme'] = 'streamflow_daily_pulse_asc'
obs.loc[obs.obsnme.str.startswith('streamflow_daily_3_3'),'obgnme'] = 'streamflow_daily_pulse_dsc'
obs.loc[obs.obsnme.str.startswith('streamflow_daily_4_1'),'obgnme'] = 'streamflow_daily_low'
obs.loc[obs.obsnme.str.startswith('streamflow_daily_5_1'),'obgnme'] = 'streamflow_daily_exlow'

#Special group for no flow
obs.loc[obs.obsnme.str.startswith('streamflow_daily_-9999_-9999'),'obgnme'] = 'streamflow_nodata'

obs.loc[obs.obsnme.str.startswith('streamflow_mon'),'obgnme'] = 'streamflow_mon'
obs.loc[obs.obsnme.str.startswith('streamflow_mean_mon'),'obgnme'] = 'streamflow_mean_mon'
obs.sample(30)

In [ ]:
obs['obgnme'].unique()

In [ ]:
#Set weights for groups"
obs.loc[obs.obgnme=='actet_mean_mon','weight'] = 3.0E+04
obs.loc[obs.obgnme=='actet_mon','weight'] = 0.75E+04
obs.loc[obs.obgnme=='recharge_ann','weight'] = 0.4E+04
obs.loc[obs.obgnme=='soil_moist_ann','weight'] = 2.5E+03
obs.loc[obs.obgnme=='soil_moist_mon','weight'] = 8E+02
obs.loc[obs.obgnme=='sca_daily','weight'] = 3E-03
obs.loc[obs.obgnme=='runoff_mon','weight'] = 7

obs.loc[obs.obgnme=='streamflow_daily_exlow','weight'] = 6
obs.loc[obs.obgnme=='streamflow_daily_low','weight'] = 1.25
obs.loc[obs.obgnme=='streamflow_daily_pulse_asc','weight'] = 0.25
obs.loc[obs.obgnme=='streamflow_daily_pulse_dsc','weight'] = 0.3
obs.loc[obs.obgnme=='streamflow_daily_small_asc','weight'] = 0.5
obs.loc[obs.obgnme=='streamflow_daily_small_dsc','weight'] = .7
obs.loc[obs.obgnme=='streamflow_daily_large_asc','weight'] = 20
obs.loc[obs.obgnme=='streamflow_daily_large_dsc','weight'] = 1

obs.loc[obs.obgnme=='streamflow_nodata','weight'] = 0

obs.loc[obs.obgnme=='streamflow_mean_mon','weight'] = 10
obs.loc[obs.obgnme=='streamflow_mon','weight'] = 2.75

obs

In [ ]:
# copy the updated obs information back to pst
pst.observation_data = obs.copy()
pst.observation_data

# consolidate the run scripts into a single script

In [ ]:
imports = [i.strip() for i in open(wkdir / 'run-pynhm.py', 'r').readlines() if i.strip().startswith('import')]
imports.extend([i.strip() for i in open('../scripts/post-process_model_output.py', 'r').readlines() if i.strip().startswith('import')])

runbiz = [i.rstrip() for i in open(wkdir / 'run-pynhm.py', 'r').readlines() if not i.strip().startswith('import')]
runbiz.append('print("#### RUN DONE, TIME TO POSTPROCESS ####")')
runbiz.extend([i.rstrip() for i in open('../scripts/post-process_model_output.py', 'r').readlines() if not i.strip().startswith('import')])


In [ ]:
runbiz

In [ ]:
# dedupe the imports
imports = list(set(imports))


### now write out all the forward run stuff

In [ ]:
with open(wkdir / 'forward_run.py', 'w') as ofp:
    [ofp.write(f'{line}\n') for line in imports+runbiz]

### and set the consolidated forward_run.py file to the pst object

In [ ]:
pst.model_command = ['python forward_run.py']

In [ ]:
pst.control_data.noptmax=0

### set some PEST++ specific parmeters

In [ ]:
pst.pestpp_options["ies_num_reals"] = 500  

pst.pestpp_options["ies_bad_phi_sigma"] = 2.5
pst.pestpp_options["overdue_giveup_fac"] = 4
pst.pestpp_options["ies_no_noise"] = True
pst.pestpp_options["ies_drop_conflicts"] = False
pst.pestpp_options["ies_pdc_sigma_distance"] = 3.0
pst.pestpp_options['ies_autoadaloc']=False
pst.pestpp_options['ies_num_threads']=4
pst.pestpp_options['ies_lambda_mults']=(0.1,1.0,10.0,100.0)
pst.pestpp_options['lambda_scale_fac'] = (0.75,0.9,1.0,1.1)
pst.pestpp_options['ies_subset_size']=10

# set SVD for some regularization
pst.svd_data.maxsing = 250

In [ ]:
pst.write(str(wkdir / 'prior_mc.pst'))

In [ ]:
pst.write(str(wkdir / 'prior_mc_better.pst'), version=2)

In [ ]:
pars


In [ ]:
pars.loc[pars.parnme.str.startswith('carea')
]

In [ ]:
pst
